In [1]:
import numpy as np

In [2]:
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import matplotlib.patches as patches
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    
    

In [4]:
#diff = np.float32(0.033068776)
diff = np.float32(0.03306878)
global_min = -31*diff
global_max = 31*diff

dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)
tau = 0.2/12600 # 0.2 times mean interparticle separation
#buffer_tau = diff*5./150.
Ngrid = 62

In [5]:
import sparkhpc

In [6]:
ncores = 8

In [7]:
import os
os.environ['SPARK_HOME'] = '/home/ics/roskar/spark'
os.environ['SPARK_DRIVER_MEMORY'] = '16G'

In [8]:
import findspark
findspark.init()

In [9]:
import sparkhpc

In [10]:
clusterid=None

if clusterid is None: 
    sj = sparkhpc.sparkjob.sparkjob(ncores=8,
                                memory=50000,
                                walltime='08:00', 
                                cores_per_executor=4, 
                                memory_per_executor=50000)
    sj.wait_to_start()
else:
    sj = sparkhpc.sparkjob.sparkjob(clusterid=clusterid)

INFO:sparkhpc.sparkjob:Submitted cluster 0


In [11]:
sj.show_clusters()

ClusterID,Job ID,Number of cores,Status,Spark UI,Spark URL
0,1169,8,running,http://172.19.1.158:8080,spark://x09y02:7077


In [12]:
sc = sparkhpc.start_spark(master=sj.master_url, spark_conf='../conf', 
                          profiling=False, executor_memory='16000M', graphframes_package='graphframes:graphframes:0.3.0-spark2.0-s_2.11')

In [13]:
sc.setCheckpointDir('file:///zbox/data/roskar/work/checkpoint')

In [14]:
import spark_fof
reload(spark_fof.spark_fof)
reload(spark_fof)
path = '/zbox/trove/euclid/2Tlc-final/'

nMinMembers = 8
nBins = 62
minblock = 30
maxblock = 34
fof_analyzer = spark_fof.spark_fof.LCFOFAnalyzer(sc, path, nMinMembers, nBins, tau, dom_mins, dom_maxs, blockids=range(minblock,maxblock), buffer_tau=tau*2)

spark_fof: Number of input files:  64
spark_fof: Total number of particles:  4695434296


In [15]:
fof_analyzer.particle_rdd.setName('particle_rdd')

particle_rdd PythonRDD[5] at RDD at PythonRDD.scala:48

In [16]:
fof_analyzer.fof_rdd.cache()

PythonRDD[13] at RDD at PythonRDD.scala:48

In [17]:
fof_analyzer.merged_rdd.cache()

spark_fof: domain group mapping build took 69.017035 seconds


merged_rdd PythonRDD[232] at RDD at PythonRDD.scala:48

In [24]:
mapping = fof_analyzer._get_level_map().cache()

spark_fof: domain group mapping build took 53.798134 seconds


In [25]:
def create_map_dict(iterator):
    group_map_dict = {g:g_p for g,g_p in iterator}
    yield group_map_dict

In [26]:
from spark_fof.spark_fof import decode_partition

In [29]:
group_merge_map = (mapping.flatMap(lambda (g,g_p):
                            [(gid, (g,g_p)) for gid in [decode_partition(g), decode_partition(g_p)]])
                          .partitionBy(8)
                          .map(lambda (k,v): v, preservesPartitioning=True)
                           .mapPartitions(create_map_dict, True)).cache()

In [30]:
fof_analyzer.group_merge_map = group_merge_map

In [20]:
merged_rdd = fof_analyzer.merged_rdd
#group_merge_map = fof_analyzer.group_merge_map
nPartitions = sc.defaultParallelism*5
nMinMembers = fof_analyzer.nMinMembers

In [21]:
from pympler import asizeof

In [ ]:
def count_groups_local(i, iterator, nMinMembers):
    # the first element is the group mapping dictionary
    dist_groups = set(iterator.next().values())
    print len(dist_groups)
    print 'sizeof set in ', i, ' ', asizeof.asizeof(dist_groups)
    group_arrs = np.concatenate([p_arr['iGroup'] for p_arr in iterator])
    gids, counts = np.unique(group_arrs, return_counts=True)
    return ((g,cnt) for (g,cnt) in zip(gids, counts) if (g in dist_groups) or (cnt >= nMinMembers))
 
def filter_groups_by_map(rdd, not_in_map=False):
    def perform_filter(iterator, exclusive):
        # the first element after the union is the group mapping
        # here we have already remapped the groups so we need to just take the final group IDs
        dist_groups = set(iterator.next().values())
        return ((gid, count) for (gid,count) in iterator if (gid in dist_groups)^exclusive)

    return rdd.mapPartitions(lambda i: perform_filter(i,not_in_map), preservesPartitioning=True)

def get_local_groups(rdd, map_rdd): 
    return filter_groups_by_map(map_rdd + rdd, not_in_map=True)

def get_distributed_groups(rdd, map_rdd):
    return filter_groups_by_map(map_rdd + rdd, not_in_map=False)

# first, get rid of ghost particles
no_ghosts_rdd = merged_rdd.map(lambda p: p[np.where(p['is_ghost'] != spark_fof.spark_fof.GHOST_PARTICLE_COPY)[0]], True)

# count up the number of particles in each group in each partition
group_counts = (group_merge_map + no_ghosts_rdd).mapPartitionsWithIndex(lambda index,i: count_groups_local(index, i, nMinMembers), True).cache()

# merge the groups that reside in multiple domains
distributed_groups = get_distributed_groups(group_counts, group_merge_map)

merge_group_counts = (distributed_groups.reduceByKey(lambda a,b: a+b, nPartitions)
                                        .filter(lambda (g,cnt): cnt>=nMinMembers)).cache()

In [16]:
%time fof_analyzer.finalize_groups()

spark_fof: domain group mapping build took 173.421325 seconds
Total number of groups:  24568111
CPU times: user 740 ms, sys: 260 ms, total: 1e+03 ms
Wall time: 1h 37min 6s


In [34]:
sj.stop()

INFO:sparkhpc.sparkjob:
